# CST 8502 FINAL PROJECT

By:
Charles-Antoine Campeau 
Joshua Ayyasamy  
Mubarak husain Shaikh  
Curtis Sloan 

Submitted to Dr. Anu Thomas in partial fulfillment of the requirements of CST 8502 

Algonquin College Artificial Intelligence Software Development

2023-11-26

In [ ]:
import pandas as pd
import numpy as np
import math
import re
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder


## DATA PREPARATION

In [ ]:
# Import the CSV
bicycleTheftData = pd.read_csv("bicycle-thefts - 4326.csv")

In [ ]:
# Remove the unecessary attributes
bicycleTheftData = bicycleTheftData.drop(columns=["_id","EVENT_UNIQUE_ID", "OCC_DATE", "OCC_YEAR", "OCC_DAY",
                                                   "OCC_DOY", "REPORT_DATE", "REPORT_YEAR", "REPORT_MONTH", "REPORT_DOW", "REPORT_DAY", 
                                                     "REPORT_DOY", "REPORT_HOUR", "LOCATION_TYPE", "BIKE_MODEL", "STATUS", "geometry"])